In [1]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [2]:
all_commits = pd.DataFrame()
all_commits_query = f"""
        SELECT DISTINCT(commits.cmt_commit_hash), repo.repo_id, repo.repo_name, repo.repo_path, 
            commits.cmt_author_email, commits.cmt_author_timestamp from repo, commits
        WHERE 
            repo.repo_id = commits.repo_id
        GROUP BY repo.repo_id, commits.cmt_commit_hash, commits.cmt_author_email, commits.cmt_author_timestamp
        ORDER BY repo.repo_id;
        """
all_commits = pd.read_sql_query(all_commits_query, con=engine)
all_commits

,cmt_commit_hash,repo_id,repo_name,repo_path,cmt_author_email,cmt_author_timestamp
0,0097eea9b9cfa0605916fac225513c14a808c910,25430,rmq-0mq,github.com/rabbitmq/,sustrik@250bpm.com,2010-09-09 08:49:09+00:00
1,04b6ed16843b336ce6ab18a693c94bbaf669dcee,25430,rmq-0mq,github.com/rabbitmq/,mikeb@squaremobius.net,2010-08-13 11:35:42+00:00
2,0a43d1cb58bab9162f50bd4c75bed5bd17bd51ae,25430,rmq-0mq,github.com/rabbitmq/,mikeb@squaremobius.net,2010-11-13 18:14:44+00:00
3,0e937a3370247b8f65e7a3eed0f66aea7f255453,25430,rmq-0mq,github.com/rabbitmq/,mikeb@squaremobius.net,2010-09-15 11:26:31+00:00
4,1436532e5e2e7381e184b52356dfb3dc3740b636,25430,rmq-0mq,github.com/rabbitmq/,mikeb@squaremobius.net,2010-09-10 16:56:12+00:00
...,...,...,...,...,...,...
1236963,e67bcfb3e20425f8c4ce9698122aa107155f5c4a,28141,boarding-pass,github.com/concourse/,suraci.alex@gmail.com,2020-01-14 00:41:15+00:00
1236964,eeae26add4a9b42101a56ad246342ecb94ee42b5,28141,boarding-pass,github.com/concourse/,suraci.alex@gmail.com,2020-01-22 14:53:45+00:00
1236965,ef8c628ff1f15455c6f46aacfe3af9f43f499b85,28141,boarding-pass,github.com/concourse/,suraci.alex@gmail.com,2020-01-14 02:29:16+00:00
1236966,f50afe58b89f7f252a0541a78593bda9e93b6537,28141,boarding-pass,github.com/concourse/,suraci.alex@gmail.com,2020-01-09 21:55:11+00:00


In [40]:
    by_repo = all_commits.loc[all_commits.groupby('repo_id').cmt_author_timestamp.idxmax()].sort_values('cmt_author_timestamp')
    
    import datetime

    current = datetime.date.today()
    start = current - datetime.timedelta(days=180)
    print(start)


2019-09-29


In [28]:
# This gives us the top contribs per repo - this can be printed to csv file 
i=0
for index, row in by_repo.iterrows():
    print(row.repo_path, row.repo_name, row.cmt_author_timestamp, row.cmt_author_email)
    print(type(row.cmt_author_timestamp))
    top_contribs = all_commits.loc[all_commits['repo_id'] == row.repo_id].cmt_author_email.value_counts()
    if len(top_contribs) > 1:
        print(top_contribs.index[0], top_contribs[0], top_contribs.index[1], top_contribs[1])
    elif len(top_contribs) == 1:
        print(top_contribs.index[0], top_contribs[0])
    elif len(top_contribs) == 0:
        print('None')
    i+=1
    if i == 50:
        break

github.com/rabbitmq/ rabbithub 2010-09-05 22:15:13+00:00 tonygarnockjones@gmail.com
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
tonyg@lshift.net 47 tonyg@kcbbs.gen.nz 23
github.com/vmware/ pyvco 2010-11-07 15:55:02+00:00 yann.hodique@gmail.com
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
yann.hodique@gmail.com 50
github.com/rabbitmq/ rabbitmq-ha 2010-12-16 14:38:01+00:00 matthew@rabbitmq.com
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
matthew@rabbitmq.com 118
github.com/Spring-projects/ cloudfoundry-sandbox 2011-06-07 15:13:23+00:00 peter@cacoethes.co.uk
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
peter@cacoethes.co.uk 1 markfisher@vmware.com 1
github.com/Spring-projects/ protocol-fallback-demo 2011-08-04 13:13:17+00:00 ozhurakousky@pivotal.io
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
ozhurakousky@pivotal.io 12 bruce.snyder@gmail.com 3
github.com/Spring-projects/ nio-sandbox 2011-08-22 13:41:24+00:00 jon@jbrisbin.com
<class 'pandas._libs.tslibs.times